In [1]:
import numpy as np
import matplotlib.pyplot as plt
from local_contributor_config import problem_folder

In [2]:
path_to_plots = r'C:/gr_latex/diploma/pictures'

Выделение инсулина на ввод пищи:
1. на белок
2. на жир
3. на глюкозу
4. суммарно

Выделение инсулина в ответ на концентрацию глюкозы в крови.  (Ввод пищи только в вену, без кишечника)

Показать график БМР. поток выходящей энергии- постоянный. При этом компоненты БМР могут меняться (во время еды, ночью и тп), а сумма выходящей энергии в единицу времени - всё рвно постоянная.